# PFRA 

### _Development Notebook_
--- 

Tests using scrapbook

In [1]:
import sys;sys.path.append('../')

In [2]:
from hecrasio.core import *
from hecrasio.qaqc import *
import scrapbook as sb
import pandas as pd
import json
import itertools

#### Read model output data 

In [3]:
# Create a large list of notebooks
nbs = ["SB-QAQC-FluvialTest.ipynb" for i in range(10)]

#### Pull scraps

In [4]:
# Enumerate here to pull it into a dictionary
scraps = {str(i)+"_"+nb: list(sb.read_notebook(nb).scraps) for i, nb in enumerate(nbs)}

In [5]:
# Pull results into a dictionary of tables
table_dict = {nb: pd.concat([pd.read_json(result) for result in results_json]).T for nb, results_json in scraps.items()}

In [6]:
# Sample results
table_dict['0_SB-QAQC-FluvialTest.ipynb']

,Base Output Interval,Computation Time Courant Method,Computation Time Step Base,Computation Time Step Count To Double,Computation Time Step Max Courant,Computation Time Step Max Doubling,Computation Time Step Max Halving,Computation Time Step Min Courant,Flow Filename,Flow Title,...,Vol Accounting 2D Bdry,Vol Accounting Cell End,Vol Accounting Cell Start,Vol Accounting DS Outflow,Vol Accounting Error,Vol Accounting Error Percentage,Vol Accounting Groundwater,"Vol Accounting LS, Pump Div",Vol Accounting Lat Hydro,Vol Accounting US Inflow
Results,15MIN,Representative Length/Velocity,30SEC,2,2,2,1,0.5,DC_F01_NBR.u01,E0004,...,0,99553.8,84515.9,0,36.8438,0.00171806,0,0,0,0


# END